# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [4]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [5]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    { 'class': 'SimpleStrategy', 'replication_factor': 1 }
""")

#### Set Keyspace

In [6]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Create Tables (song_session, user_session, and user_song)

In [7]:
##################
## Table:       music_library_by_sessions
## Description: This table gives the Music Library by Sessions. 
##              eg.: We can keep track of the specific
##              session item that a user listened to a song,
##              and for how long they listened it to.
##
## Primary Key: sessionId (PARTITION KEY) and iteminsession (CLUSTERING COLUMN)
##
##################
query = "CREATE TABLE IF NOT EXISTS music_library_by_sessions "
query = query + """(sessionid int, \n
                    iteminsession int, \n
                    artist text, \n                    
                    song text, \n
                    length float, \n
                    PRIMARY KEY (sessionid, iteminsession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)
    
##################
## Table:       music_library_by_users
## Description: This table gives the Music Library by Users. 
##              eg.: We can keep track of the session
##              that a user listened to a song.
##
## Primary Key: userid, sessionid (PARTITION KEY) and iteminsession (CLUSTERING COLUMN)
##
##################
query = "CREATE TABLE IF NOT EXISTS music_library_by_users "
query = query + """(userid int,
                    sessionid int, \n
                    iteminsession int, \n
                    artist text, \n
                    song text, \n
                    firstname text, \n
                    lastname text, \n
                    PRIMARY KEY ((userid, sessionid), iteminsession))"""
try:
    session.execute(query)
except Exception as e:
    print(e)
    
##################
## Table:       users_by_music
## Description: This table gives all the Users by a given Music. 
##              eg.: We can keep track of the specific
##              users (first name and last name)
##              that listened to a specific song.
##
## Primary Key: song (PARTITION KEY) and userid (CLUSTERING COLUMN)
##
##################
query = "CREATE TABLE IF NOT EXISTS users_by_music "
query = query + """(userid int, \n
                    firstname text, \n                    
                    lastname text, \n
                    song text, \n
                    PRIMARY KEY (song, userid))"""
try:
    session.execute(query)
except Exception as e:
    print(e)

# Insert entries from CSV into assigned TABLES

In [8]:
file = 'event_datafile_new.csv'

### COLUMN MAPPING            
# session.execute(query, ( 
#     line[0],          # artist
#     line[1],          # firstName
#     line[2],          # gender
#     int(line[3]),     # itemInSession
#     line[4],          # lastName
#     float(line[5]),   # length
#     line[6],          # level
#     line[7],          # location              
#     int(line[8]),     # sessionId
#     line[9],          # song
#     int(line[10])     # userId
# ))

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            query1 = """INSERT INTO music_library_by_sessions ( \n
                                    sessionid, \n
                                    iteminsession, \n
                                    artist, \n
                                    song, \n
                                    length                                
                    )"""
            query1 = query1 + " VALUES (%s, %s, %s, %s, %s)"
            session.execute(query1, (
                int(line[8]),
                int(line[3]),
                line[0],
                line[9],
                float(line[5])
            ))
            
            query2 = """INSERT INTO music_library_by_users ( \n
                                    userid, \n
                                    sessionid, \n
                                    iteminsession, \n
                                    artist, \n
                                    song, \n
                                    firstname, \n
                                    lastname
                    )"""
            query2 = query2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
            session.execute(query2, (
                int(line[10]), 
                int(line[8]),
                int(line[3]),
                line[0],
                line[9],
                line[1],
                line[4]
            ))
            
            query3 = """INSERT INTO users_by_music ( \n           
                                    userid, \n
                                    firstname, \n
                                    lastname, \n
                                    song
                    )"""
            query3 = query3 + " VALUES (%s, %s, %s, %s)"
            session.execute(query3, (                
                int(line[10]),
                line[1],                
                line[4],
                line[9]
            ))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [9]:
query1 = "SELECT sessionid, iteminsession, artist, song, length FROM music_library_by_sessions WHERE sessionid=23 AND iteminsession=1;"
try:
    rows1 = session.execute(query1)    
except Exception as e:
    print(e)
    
query2 = "SELECT userid, sessionid, iteminsession, artist, firstname, lastname, song, song FROM music_library_by_users WHERE userid=58 AND sessionid=768 LIMIT 5;"
try:
    rows2 = session.execute(query2)    
except Exception as e:
    print(e)
    
query3 = "SELECT userid, firstname, lastname, song FROM users_by_music WHERE song='In The Dragon''s Den' AND userid=49 LIMIT 5;"
try:
    rows3 = session.execute(query3)    
except Exception as e:
    print(e)

print('========= query1 =========')
for row in rows1:
    print(row)
print('========= query2 =========')
for row in rows2:
    print(row)
print('========= query3 =========')
for row in rows3:
    print(row)

========= query1 =========
Row(sessionid=23, iteminsession=1, artist='Octopus Project', song='All Of The Champs That Ever Lived', length=250.95791625976562)
========= query2 =========
Row(userid=58, sessionid=768, iteminsession=0, artist='System of a Down', firstname='Emily', lastname='Benson', song='Sad Statue', song_='Sad Statue')
Row(userid=58, sessionid=768, iteminsession=1, artist='Ghostland Observatory', firstname='Emily', lastname='Benson', song='Stranger Lover', song_='Stranger Lover')
Row(userid=58, sessionid=768, iteminsession=2, artist='Evergreen Terrace', firstname='Emily', lastname='Benson', song='Zero', song_='Zero')
========= query3 =========
Row(userid=49, firstname='Chloe', lastname='Cuevas', song="In The Dragon's Den")


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

**1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4**

In [65]:
query = "SELECT artist, song, length FROM music_library_by_sessions WHERE iteminsession=4 AND sessionid=338"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = pd.DataFrame(rows._current_rows)
df

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


**2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182**

In [66]:
query = "SELECT artist, song, firstname, lastname FROM music_library_by_users WHERE userid=10 AND sessionid=182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
df = pd.DataFrame(rows._current_rows)
df

,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


**3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'**

In [67]:
query = "SELECT firstname, lastname FROM users_by_music WHERE song='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

df = pd.DataFrame(rows._current_rows)
df

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


In [54]:
query = "DROP TABLE music_library_by_sessions"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE music_library_by_users"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE users_by_music"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()